In [ ]:
from kan import *
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from sklearn.datasets import make_moons
import pandas as pd
from sklearn.model_selection import train_test_split

np.random.seed(0)

### Usando fórmula

In [ ]:
dataset = {}
train_input, train_label = make_moons(n_samples=1000, shuffle=True, noise=0.1, random_state=0)
test_input, test_label = make_moons(n_samples=1000, shuffle=True, noise=0.1, random_state=0)

dtype = torch.get_default_dtype()
dataset['train_input'] = torch.from_numpy(train_input).type(dtype)
dataset['test_input'] = torch.from_numpy(test_input).type(dtype)
dataset['train_label'] = torch.from_numpy(train_label[:, None]).type(dtype)
dataset['test_label'] = torch.from_numpy(test_label[:, None]).type(dtype)

X = dataset['train_input']
y = dataset['train_label']
plt.figure()
plt.scatter(dataset['train_input'][:, 0], dataset['train_input'][:, 1], c=dataset['train_label'])
plt.title('TRAIN')
plt.show()
plt.figure()
plt.scatter(dataset['test_input'][:, 0], dataset['test_input'][:, 1], c=dataset['test_label'])
plt.title('TEST')
plt.show()

In [ ]:
# [2,5,1] means 2 inputs, 5 hidden add neurons, and 1 output
# [2,[5,2],1] means 2 inputs, 5 hidden add neurons and 2 hidden multiplication neurons, and 1 output.
model = KAN(width=[2, 5, 3, 2, 1, 1], grid=3, k=3)


model_results = model.fit(dataset, opt="LBFGS", steps=100)

In [ ]:
# model.plot()  # semi quebrado na 0.2.1

In [ ]:
lib = ['x', 'x^2', 'x^3', 'x^4', 'exp', 'log', 'sqrt', 'tanh', 'sin', 'tan', 'abs', 'cos']
model.auto_symbolic(lib=lib)  # semi quebrado na 0.2.1: pesa muito o c, se nao passar a lib quase tudo vira 0
formula = model.symbolic_formula()[0][0]

In [ ]:
formula

In [ ]:
expr_func = sp.lambdify(['x_1', 'x_2'], formula, "numpy")
results = expr_func(dataset['test_input'][:, 0], dataset['test_input'][:, 1])
plt.figure()
plt.scatter(dataset['test_input'][:, 0], dataset['test_input'][:, 1], c=results)  # np.round(results,0)
plt.title('PREDICTION')
plt.show()

### Usando Dataset

In [ ]:
df = pd.read_csv('winequality-red.csv', sep=";")
df

In [ ]:
X = df.drop(columns=['quality'])
y = df['quality']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
dtype = torch.get_default_dtype()

dataset = {}
dataset['train_input'] = torch.from_numpy(X_train.values).type(dtype)
dataset['test_input'] = torch.from_numpy(X_test.values).type(dtype)

dataset['train_label'] = torch.from_numpy(y_train.values).type(dtype)
dataset['test_label'] = torch.from_numpy(y_test.values).type(dtype)

In [ ]:
model = KAN(width=[X_train.shape[1], 6, 5, 2, 1], grid=3, k=3)


model_results = model.fit(dataset, opt="LBFGS", steps=100)

In [ ]:
lib = ['x', 'x^2', 'x^3', 'x^4', 'exp', 'log', 'sqrt', 'tanh', 'sin', 'tan', 'abs', 'cos']
model.auto_symbolic(lib=lib)  # semi quebrado na 0.2.1: pesa muito o c, se nao passar a lib quase tudo vira 0
formula = model.symbolic_formula()[0][0]

In [ ]:
formula

In [ ]:
results = expr_func(**{f'x_{i+1}': np.array(dataset['test_input'][:, i]) for i in range(X_test.shape[1])})

In [ ]:
plt.figure()
plt.scatter(np.arange(y_test.shape[0]), y_test)  # np.round(results,0)
plt.scatter(np.arange(y_test.shape[0]), results)
plt.title('PREDICTION')
plt.show()

# Comparando com MLP